### Capstone 2- Bengali Grapheme Classification Project

This workbook is the first attempt at experimenting with a CNN for this project. I've followed this [starter code] (https://www.kaggle.com/kaushal2896/bengali-graphemes-starter-eda-multi-output-cnn)for ideas. 

This is also the first attempt at using MLFlow for tracking deep learning experiments

In [ ]:
#importing necessary packages
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import sklearn
from collections import defaultdict
import mlflow

Although my virtual environment uses tensorflow-gpu, I want to doubly make sure that keras uses my GPU

In [ ]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

In [ ]:
# using custom scripts for creating model and loading data
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, './model/')

In [ ]:
from model_creator import model_create
from data_loader import data_loader

In [ ]:
# read the csv files
filenames = ['train','test','class_map','class_map_corrected','train_multi_diacritics','sample_submission']
df_dict = defaultdict()

for file in filenames:
    df_dict[file]=pd.read_csv('./data/{}.csv'.format(file))

In [ ]:
# using mlflow autologger to track models, artifacts and parameters
import mlflow.keras
mlflow.keras.autolog()

In [ ]:
#creating mlflow experiment
mlflow.create_experiment(name="conv_kernel_size_6_pool_size_3_default_arch")

In [ ]:
#this version of the model create function creates a 3 layer alternating conv-maxpool layers followed by dropout
#two dense layers with a dropout in the middle and three output layers

#to-do batch normalization, add optimizer and initializer as parameters
model = model_create(conv_kernel_size=6,pool_size=3)

In [ ]:
#view summary of model created
model.summary()

In [ ]:
#specify hyper-parameters
batch_size = 256
epochs = 50

In [ ]:
#note - this implementation did not save experiment under the experiment name and instead saved it in the default name

#to do - investigate why
with mlflow.start_run():
    for i in range(4):
        #iterate through all 4 training parquet files 
        print('Reading parquet file #{}'.format(i+1))
        print('---------------------------------------')
        print('Transforming data for parquet file #{}'.format(i+1))
        print('---------------------------------------')
        x_train, x_test, y_train_root, y_test_root, y_train_consonant, y_test_consonant, y_train_vowel, y_test_vowel=data_loader('./data/train_image_data_{}.parquet'.format(i),df_dict['train'])
        print('Training model on parquet file #{}'.format(i+1))
        print('---------------------------------------')
        history=model.fit(x=x_train, y={"output_root":y_train_root,
                                    "output_vowel":y_train_vowel,"output_consonant":y_train_consonant},
                      epochs=epochs,batch_size=batch_size,
                      validation_data=(x_test,{"output_root":y_test_root,"output_vowel":y_test_vowel,
                                               "output_consonant":y_test_consonant}))
        print('Deleting variables after training')
        del x_train, x_test, y_train_root, y_test_root, y_train_consonant, y_test_consonant, y_train_vowel, y_test_vowel


In [ ]:
# getting predictions on test set
# this code is borrowed from the started code mentioned at the top but modified for my implementation

preds_dict = {
    'grapheme_root': [],
    'vowel_diacritic': [],
    'consonant_diacritic': []
}

components = ['consonant_diacritic', 'grapheme_root', 'vowel_diacritic']
target=[] # model predictions placeholder
row_id=[] # row_id place holder
for i in range(4):
    df_test_img = pd.read_parquet('./data/test_image_data_{}.parquet'.format(i)) 
    df_test_img.set_index('image_id', inplace=True)


    X_test = df_test_img.values.reshape(-1, 137, 236, 1)
    
    preds = model.predict(X_test)

    for i, p in enumerate(preds_dict):
        preds_dict[p] = np.argmax(preds[i], axis=1)

    for k,id in enumerate(df_test_img.index.values):  
        for i,comp in enumerate(components):
            id_sample=id+'_'+comp
            row_id.append(id_sample)
            target.append(preds_dict[comp][k])
    del df_test_img
    del X_test


df_sample = pd.DataFrame(
    {
        'row_id': row_id,
        'target':target
    },
    columns = ['row_id','target'] 
)
df_sample.to_csv('submission.csv',index=False)
df_sample


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import sklearn
from collections import defaultdict
import mlflow

In [ ]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

In [ ]:
# using custom scripts for creating model and loading data
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, './model/')

In [ ]:
from model_creator import model_create
from data_loader import data_loader

In [ ]:
# read the csv files
filenames = ['train','test','class_map','class_map_corrected','train_multi_diacritics','sample_submission']
df_dict = defaultdict()

for file in filenames:
    df_dict[file]=pd.read_csv('./data/{}.csv'.format(file))

In [ ]:
#specify hyper-parameters
batch_size = 256
epochs = 50

In [ ]:
import mlflow.keras
mlflow.create_experiment(name="conv_kernel_size_6_pool_size_3_batch_norm")
mlflow.keras.autolog()

with mlflow.start_run():
    model = model_create(conv_kernel_size=6,pool_size=3)
    for i in range(4):
        #iterate through all 4 training parquet files 
        print('Reading parquet file #{}'.format(i+1))
        print('---------------------------------------')
        print('Transforming data for parquet file #{}'.format(i+1))
        print('---------------------------------------')
        x_train, x_test, y_train_root, y_test_root, y_train_consonant, y_test_consonant, y_train_vowel, y_test_vowel=data_loader('./data/train_image_data_{}.parquet'.format(i),df_dict['train'])
        print('Training model on parquet file #{}'.format(i+1))
        print('---------------------------------------')
        history=model.fit(x=x_train, y={"output_root":y_train_root,
                                    "output_vowel":y_train_vowel,"output_consonant":y_train_consonant},
                      epochs=epochs,batch_size=batch_size,
                      validation_data=(x_test,{"output_root":y_test_root,"output_vowel":y_test_vowel,
                                               "output_consonant":y_test_consonant}))
        print('Deleting variables after training')
        del x_train, x_test, y_train_root, y_test_root, y_train_consonant, y_test_consonant, y_train_vowel, y_test_vowel


In [ ]:
# getting predictions on test set
# this code is borrowed from the started code mentioned at the top but modified for my implementation

preds_dict = {
    'grapheme_root': [],
    'vowel_diacritic': [],
    'consonant_diacritic': []
}

components = ['consonant_diacritic', 'grapheme_root', 'vowel_diacritic']
target=[] # model predictions placeholder
row_id=[] # row_id place holder
for i in range(4):
    df_test_img = pd.read_parquet('./data/test_image_data_{}.parquet'.format(i)) 
    df_test_img.set_index('image_id', inplace=True)


    X_test = df_test_img.values.reshape(-1, 137, 236, 1)
    
    preds = model.predict(X_test)

    for i, p in enumerate(preds_dict):
        preds_dict[p] = np.argmax(preds[i], axis=1)

    for k,id in enumerate(df_test_img.index.values):  
        for i,comp in enumerate(components):
            id_sample=id+'_'+comp
            row_id.append(id_sample)
            target.append(preds_dict[comp][k])
    del df_test_img
    del X_test


df_sample = pd.DataFrame(
    {
        'row_id': row_id,
        'target':target
    },
    columns = ['row_id','target'] 
)
df_sample.to_csv('submission.csv',index=False)
df_sample


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import sklearn
from collections import defaultdict
import mlflow

In [ ]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

In [ ]:
import mlflow.keras
mlflow.create_experiment("conv_kernel_size_6_pool_size_3_batch_norm_dropout_rate_0-5")
mlflow.set_experiment("conv_kernel_size_6_pool_size_3_batch_norm_dropout_rate_0-5")

mlflow.keras.autolog()

with mlflow.start_run():
    model = model_create(conv_kernel_size=6,pool_size=3,dropout_rate1=0.5,dropout_rate2=0.5)
    for i in range(4):
        #iterate through all 4 training parquet files 
        print('Reading parquet file #{}'.format(i+1))
        print('---------------------------------------')
        print('Transforming data for parquet file #{}'.format(i+1))
        print('---------------------------------------')
        x_train, x_test, y_train_root, y_test_root, y_train_consonant, y_test_consonant, y_train_vowel, y_test_vowel=data_loader('./data/train_image_data_{}.parquet'.format(i),df_dict['train'])
        print('Training model on parquet file #{}'.format(i+1))
        print('---------------------------------------')
        history=model.fit(x=x_train, y={"output_root":y_train_root,
                                    "output_vowel":y_train_vowel,"output_consonant":y_train_consonant},
                      epochs=50,batch_size=256,
                      validation_data=(x_test,{"output_root":y_test_root,"output_vowel":y_test_vowel,
                                               "output_consonant":y_test_consonant}))
        print('Deleting variables after training')
        del x_train, x_test, y_train_root, y_test_root, y_train_consonant, y_test_consonant, y_train_vowel, y_test_vowel


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import sklearn
from collections import defaultdict
import mlflow
import random
random.seed(42)
# using custom scripts for creating model and loading data
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, './model/')

from model_creator import model_create
from data_loader import data_loader

In [ ]:
from experiments import run_experiment
from tester import test_func

In [ ]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

In [ ]:
experiments = {1:{'conv_kernel_size':6,'n_conv_layers':3,'pool_size':3,'dropout_rate':0.5},
               2:{'conv_kernel_size':6,'n_conv_layers':3,'pool_size':3,'dropout_rate':0.3},
               3:{'conv_kernel_size':6,'n_conv_layers':4,'pool_size':3,'dropout_rate':0.3},
               4:{'conv_kernel_size':10,'n_conv_layers':3,'pool_size':5,'dropout_rate':0.3},
        }

In [ ]:
experiments

In [ ]:
len(experiments)

In [ ]:
# read the csv files
filenames = ['train','test','class_map','class_map_corrected','train_multi_diacritics','sample_submission']
df_dict = defaultdict()

for file in filenames:
    df_dict[file]=pd.read_csv('./data/{}.csv'.format(file))

In [ ]:
experiments[1]['conv_kernel_size']

In [ ]:
titles={1:"conv_layers_3_kernel_size_6_pool_3_dropout_0.5",2:"conv_layers_3_kernel_size_6_pool_3_dropout_0.3",
       3:"conv_layers_4_kernel_size_6_pool_3_dropout_0.3",4:"conv_layers_3_kernel_size_10_pool_5_dropout_0.3"}

In [ ]:
import mlflow.keras
for i in range(len(experiments)):
    model = model_create(conv_kernel_size=experiments[i+1]['conv_kernel_size'],pool_size=experiments[i+1]['pool_size'],
                        dropout_rate1=experiments[i+1]['dropout_rate'],dropout_rate2=experiments[i+1]['dropout_rate'],
                        n_conv_layers=experiments[i+1]['n_conv_layers'])
    
    print('Running Experiment-{}'.format(i+1))
    print('---------------------------------------------------------------------')
    run_experiment(titles[i+1],model,batch_size=256,epochs=50,df_dict=df_dict)

print('All experiments finished')

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import sklearn
from collections import defaultdict
import mlflow
import random
random.seed(42)
# using custom scripts for creating model and loading data
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, './model/')

from model_creator import model_create
from data_loader import data_loader

from experiments import run_experiment
from tester import test_func

from batch_generator import MultiOutputDataGenerator

from keras import backend as K
K.tensorflow_backend._get_available_gpus()

Using TensorFlow backend.


['/job:localhost/replica:0/task:0/device:GPU:0']

In [2]:
experiments = {1:{'conv_kernel_size':3,'n_conv_layers':5,'pool_size':2,'dropout_rate':0.1,'conv_nfilters':100},
               2:{'conv_kernel_size':3,'n_conv_layers':5,'pool_size':2,'dropout_rate':0.25,'conv_nfilters':100},
               3:{'conv_kernel_size':3,'n_conv_layers':5,'pool_size':2,'dropout_rate':0.5,'conv_nfilters':100},
               4:{'conv_kernel_size':3,'n_conv_layers':5,'pool_size':2,'dropout_rate':0.75,'conv_nfilters':100},
               5:{'conv_kernel_size':4,'n_conv_layers':5,'pool_size':2,'dropout_rate':0.1,'conv_nfilters':100},
               6:{'conv_kernel_size':4,'n_conv_layers':5,'pool_size':2,'dropout_rate':0.25,'conv_nfilters':100},
               7:{'conv_kernel_size':4,'n_conv_layers':5,'pool_size':2,'dropout_rate':0.5,'conv_nfilters':100},
               8:{'conv_kernel_size':4,'n_conv_layers':5,'pool_size':2,'dropout_rate':0.75,'conv_nfilters':100},
               9:{'conv_kernel_size':5,'n_conv_layers':5,'pool_size':2,'dropout_rate':0.1,'conv_nfilters':100},
               10:{'conv_kernel_size':5,'n_conv_layers':5,'pool_size':2,'dropout_rate':0.25,'conv_nfilters':100},
               11:{'conv_kernel_size':5,'n_conv_layers':5,'pool_size':2,'dropout_rate':0.5,'conv_nfilters':100},
               12:{'conv_kernel_size':5,'n_conv_layers':5,'pool_size':2,'dropout_rate':0.75,'conv_nfilters':100},
               13:{'conv_kernel_size':3,'n_conv_layers':4,'pool_size':2,'dropout_rate':0.1,'conv_nfilters':100},
               14:{'conv_kernel_size':3,'n_conv_layers':4,'pool_size':2,'dropout_rate':0.25,'conv_nfilters':100},
               15:{'conv_kernel_size':3,'n_conv_layers':4,'pool_size':2,'dropout_rate':0.5,'conv_nfilters':100},
               16:{'conv_kernel_size':3,'n_conv_layers':4,'pool_size':2,'dropout_rate':0.75,'conv_nfilters':100},
               17:{'conv_kernel_size':4,'n_conv_layers':4,'pool_size':2,'dropout_rate':0.1,'conv_nfilters':100},
               18:{'conv_kernel_size':4,'n_conv_layers':4,'pool_size':2,'dropout_rate':0.25,'conv_nfilters':100},
               19:{'conv_kernel_size':4,'n_conv_layers':4,'pool_size':2,'dropout_rate':0.5,'conv_nfilters':100},
               20:{'conv_kernel_size':4,'n_conv_layers':4,'pool_size':2,'dropout_rate':0.75,'conv_nfilters':100},
               21:{'conv_kernel_size':5,'n_conv_layers':4,'pool_size':2,'dropout_rate':0.1,'conv_nfilters':100},
               22:{'conv_kernel_size':5,'n_conv_layers':4,'pool_size':2,'dropout_rate':0.25,'conv_nfilters':100},
               23:{'conv_kernel_size':5,'n_conv_layers':4,'pool_size':2,'dropout_rate':0.5,'conv_nfilters':100},
               24:{'conv_kernel_size':5,'n_conv_layers':4,'pool_size':2,'dropout_rate':0.75,'conv_nfilters':100},
               25:{'conv_kernel_size':3,'n_conv_layers':3,'pool_size':2,'dropout_rate':0.1,'conv_nfilters':100},
               26:{'conv_kernel_size':3,'n_conv_layers':3,'pool_size':2,'dropout_rate':0.25,'conv_nfilters':100},
               27:{'conv_kernel_size':3,'n_conv_layers':3,'pool_size':2,'dropout_rate':0.5,'conv_nfilters':100},
               28:{'conv_kernel_size':3,'n_conv_layers':3,'pool_size':2,'dropout_rate':0.75,'conv_nfilters':100},
               29:{'conv_kernel_size':4,'n_conv_layers':3,'pool_size':2,'dropout_rate':0.1,'conv_nfilters':100},
               30:{'conv_kernel_size':4,'n_conv_layers':3,'pool_size':2,'dropout_rate':0.25,'conv_nfilters':100},
               31:{'conv_kernel_size':4,'n_conv_layers':3,'pool_size':2,'dropout_rate':0.5,'conv_nfilters':100},
               32:{'conv_kernel_size':4,'n_conv_layers':3,'pool_size':2,'dropout_rate':0.75,'conv_nfilters':100},
               33:{'conv_kernel_size':5,'n_conv_layers':3,'pool_size':2,'dropout_rate':0.1,'conv_nfilters':100},
               34:{'conv_kernel_size':5,'n_conv_layers':3,'pool_size':2,'dropout_rate':0.25,'conv_nfilters':100},
               35:{'conv_kernel_size':5,'n_conv_layers':3,'pool_size':2,'dropout_rate':0.5,'conv_nfilters':100},
               36:{'conv_kernel_size':5,'n_conv_layers':3,'pool_size':2,'dropout_rate':0.75,'conv_nfilters':100},
               37:{'conv_kernel_size':3,'n_conv_layers':5,'pool_size':2,'dropout_rate':0.1,'conv_nfilters':75},
               38:{'conv_kernel_size':3,'n_conv_layers':5,'pool_size':2,'dropout_rate':0.25,'conv_nfilters':75},
               39:{'conv_kernel_size':3,'n_conv_layers':5,'pool_size':2,'dropout_rate':0.5,'conv_nfilters':75},
               40:{'conv_kernel_size':3,'n_conv_layers':5,'pool_size':2,'dropout_rate':0.75,'conv_nfilters':75},
               41:{'conv_kernel_size':4,'n_conv_layers':5,'pool_size':2,'dropout_rate':0.1,'conv_nfilters':75},
               42:{'conv_kernel_size':4,'n_conv_layers':5,'pool_size':2,'dropout_rate':0.25,'conv_nfilters':75},
               43:{'conv_kernel_size':4,'n_conv_layers':5,'pool_size':2,'dropout_rate':0.5,'conv_nfilters':75},
               44:{'conv_kernel_size':4,'n_conv_layers':5,'pool_size':2,'dropout_rate':0.75,'conv_nfilters':75},
               45:{'conv_kernel_size':5,'n_conv_layers':5,'pool_size':2,'dropout_rate':0.1,'conv_nfilters':75},
               46:{'conv_kernel_size':5,'n_conv_layers':5,'pool_size':2,'dropout_rate':0.25,'conv_nfilters':75},
               47:{'conv_kernel_size':5,'n_conv_layers':5,'pool_size':2,'dropout_rate':0.5,'conv_nfilters':75},
               48:{'conv_kernel_size':5,'n_conv_layers':5,'pool_size':2,'dropout_rate':0.75,'conv_nfilters':75},
               49:{'conv_kernel_size':3,'n_conv_layers':4,'pool_size':2,'dropout_rate':0.1,'conv_nfilters':75},
               50:{'conv_kernel_size':3,'n_conv_layers':4,'pool_size':2,'dropout_rate':0.25,'conv_nfilters':75},
               51:{'conv_kernel_size':3,'n_conv_layers':4,'pool_size':2,'dropout_rate':0.5,'conv_nfilters':75},
               52:{'conv_kernel_size':3,'n_conv_layers':4,'pool_size':2,'dropout_rate':0.75,'conv_nfilters':75},
               53:{'conv_kernel_size':4,'n_conv_layers':4,'pool_size':2,'dropout_rate':0.1,'conv_nfilters':75},
               54:{'conv_kernel_size':4,'n_conv_layers':4,'pool_size':2,'dropout_rate':0.25,'conv_nfilters':75},
               55:{'conv_kernel_size':4,'n_conv_layers':4,'pool_size':2,'dropout_rate':0.5,'conv_nfilters':75},
               56:{'conv_kernel_size':4,'n_conv_layers':4,'pool_size':2,'dropout_rate':0.75,'conv_nfilters':75},
               57:{'conv_kernel_size':5,'n_conv_layers':4,'pool_size':2,'dropout_rate':0.1,'conv_nfilters':75},
               58:{'conv_kernel_size':5,'n_conv_layers':4,'pool_size':2,'dropout_rate':0.25,'conv_nfilters':75},
               59:{'conv_kernel_size':5,'n_conv_layers':4,'pool_size':2,'dropout_rate':0.5,'conv_nfilters':75},
               60:{'conv_kernel_size':5,'n_conv_layers':4,'pool_size':2,'dropout_rate':0.75,'conv_nfilters':75},
               61:{'conv_kernel_size':3,'n_conv_layers':3,'pool_size':2,'dropout_rate':0.1,'conv_nfilters':75},
               62:{'conv_kernel_size':3,'n_conv_layers':3,'pool_size':2,'dropout_rate':0.25,'conv_nfilters':75},
               63:{'conv_kernel_size':3,'n_conv_layers':3,'pool_size':2,'dropout_rate':0.5,'conv_nfilters':75},
               64:{'conv_kernel_size':3,'n_conv_layers':3,'pool_size':2,'dropout_rate':0.75,'conv_nfilters':75},
               65:{'conv_kernel_size':4,'n_conv_layers':3,'pool_size':2,'dropout_rate':0.1,'conv_nfilters':75},
               66:{'conv_kernel_size':4,'n_conv_layers':3,'pool_size':2,'dropout_rate':0.25,'conv_nfilters':75},
               67:{'conv_kernel_size':4,'n_conv_layers':3,'pool_size':2,'dropout_rate':0.5,'conv_nfilters':75},
               68:{'conv_kernel_size':4,'n_conv_layers':3,'pool_size':2,'dropout_rate':0.75,'conv_nfilters':75},
               69:{'conv_kernel_size':5,'n_conv_layers':3,'pool_size':2,'dropout_rate':0.1,'conv_nfilters':75},
               70:{'conv_kernel_size':5,'n_conv_layers':3,'pool_size':2,'dropout_rate':0.25,'conv_nfilters':75},
               71:{'conv_kernel_size':5,'n_conv_layers':3,'pool_size':2,'dropout_rate':0.5,'conv_nfilters':75},
               72:{'conv_kernel_size':5,'n_conv_layers':3,'pool_size':2,'dropout_rate':0.75,'conv_nfilters':75},
               73:{'conv_kernel_size':3,'n_conv_layers':5,'pool_size':2,'dropout_rate':0.1,'conv_nfilters':50},
               74:{'conv_kernel_size':3,'n_conv_layers':5,'pool_size':2,'dropout_rate':0.25,'conv_nfilters':50},
               75:{'conv_kernel_size':3,'n_conv_layers':5,'pool_size':2,'dropout_rate':0.5,'conv_nfilters':50},
               76:{'conv_kernel_size':3,'n_conv_layers':5,'pool_size':2,'dropout_rate':0.75,'conv_nfilters':50},
               77:{'conv_kernel_size':4,'n_conv_layers':5,'pool_size':2,'dropout_rate':0.1,'conv_nfilters':50},
               78:{'conv_kernel_size':4,'n_conv_layers':5,'pool_size':2,'dropout_rate':0.25,'conv_nfilters':50},
               79:{'conv_kernel_size':4,'n_conv_layers':5,'pool_size':2,'dropout_rate':0.5,'conv_nfilters':50},
               80:{'conv_kernel_size':4,'n_conv_layers':5,'pool_size':2,'dropout_rate':0.75,'conv_nfilters':50},
               81:{'conv_kernel_size':5,'n_conv_layers':5,'pool_size':2,'dropout_rate':0.1,'conv_nfilters':50},
               82:{'conv_kernel_size':5,'n_conv_layers':5,'pool_size':2,'dropout_rate':0.25,'conv_nfilters':50},
               83:{'conv_kernel_size':5,'n_conv_layers':5,'pool_size':2,'dropout_rate':0.5,'conv_nfilters':50},
               84:{'conv_kernel_size':5,'n_conv_layers':5,'pool_size':2,'dropout_rate':0.75,'conv_nfilters':50},
               85:{'conv_kernel_size':3,'n_conv_layers':4,'pool_size':2,'dropout_rate':0.1,'conv_nfilters':50},
               86:{'conv_kernel_size':3,'n_conv_layers':4,'pool_size':2,'dropout_rate':0.25,'conv_nfilters':50},
               87:{'conv_kernel_size':3,'n_conv_layers':4,'pool_size':2,'dropout_rate':0.5,'conv_nfilters':50},
               88:{'conv_kernel_size':3,'n_conv_layers':4,'pool_size':2,'dropout_rate':0.75,'conv_nfilters':50},
               89:{'conv_kernel_size':4,'n_conv_layers':4,'pool_size':2,'dropout_rate':0.1,'conv_nfilters':50},
               90:{'conv_kernel_size':4,'n_conv_layers':4,'pool_size':2,'dropout_rate':0.25,'conv_nfilters':50},
               91:{'conv_kernel_size':4,'n_conv_layers':4,'pool_size':2,'dropout_rate':0.5,'conv_nfilters':50},
               92:{'conv_kernel_size':4,'n_conv_layers':4,'pool_size':2,'dropout_rate':0.75,'conv_nfilters':50},
               93:{'conv_kernel_size':5,'n_conv_layers':4,'pool_size':2,'dropout_rate':0.1,'conv_nfilters':50},
               94:{'conv_kernel_size':5,'n_conv_layers':4,'pool_size':2,'dropout_rate':0.25,'conv_nfilters':50},
               95:{'conv_kernel_size':5,'n_conv_layers':4,'pool_size':2,'dropout_rate':0.5,'conv_nfilters':50},
               96:{'conv_kernel_size':5,'n_conv_layers':4,'pool_size':2,'dropout_rate':0.75,'conv_nfilters':50},
               97:{'conv_kernel_size':3,'n_conv_layers':3,'pool_size':2,'dropout_rate':0.1,'conv_nfilters':50},
               98:{'conv_kernel_size':3,'n_conv_layers':3,'pool_size':2,'dropout_rate':0.25,'conv_nfilters':50},
               99:{'conv_kernel_size':3,'n_conv_layers':3,'pool_size':2,'dropout_rate':0.5,'conv_nfilters':50},
               100:{'conv_kernel_size':3,'n_conv_layers':3,'pool_size':2,'dropout_rate':0.75,'conv_nfilters':50},
               101:{'conv_kernel_size':4,'n_conv_layers':3,'pool_size':2,'dropout_rate':0.1,'conv_nfilters':50},
               102:{'conv_kernel_size':4,'n_conv_layers':3,'pool_size':2,'dropout_rate':0.25,'conv_nfilters':50},
               103:{'conv_kernel_size':4,'n_conv_layers':3,'pool_size':2,'dropout_rate':0.5,'conv_nfilters':50},
               104:{'conv_kernel_size':4,'n_conv_layers':3,'pool_size':2,'dropout_rate':0.75,'conv_nfilters':50},
               105:{'conv_kernel_size':5,'n_conv_layers':3,'pool_size':2,'dropout_rate':0.1,'conv_nfilters':50},
               106:{'conv_kernel_size':5,'n_conv_layers':3,'pool_size':2,'dropout_rate':0.25,'conv_nfilters':50},
               107:{'conv_kernel_size':5,'n_conv_layers':3,'pool_size':2,'dropout_rate':0.5,'conv_nfilters':50},
               108:{'conv_kernel_size':5,'n_conv_layers':3,'pool_size':2,'dropout_rate':0.75,'conv_nfilters':50},
        }

In [3]:
titles=dict()
for i in range(0,len(experiments)):
    titles[i+1]="compress_exp_aug_1_conv_layers_{0}_conv_kernel_size_{1}_pool_size_{2}_dropout_rate_{3}_nfilters_{4}".format(
        experiments[i+1]['n_conv_layers'],experiments[i+1]['conv_kernel_size'],experiments[i+1]['pool_size'],
    experiments[i+1]['dropout_rate'],experiments[i+1]['conv_nfilters'])

In [4]:
# read the csv files
filenames = ['train','test','class_map','class_map_corrected','train_multi_diacritics','sample_submission']
df_dict = defaultdict()

for file in filenames:
    df_dict[file]=pd.read_csv('./data/{}.csv'.format(file))

In [ ]:
import mlflow.keras
modelerrCount=0
experrCount=0
for i in range(0,len(experiments)):
    try:
        print('Creating Model')
        model = model_create(input_shape=(64,37,1),conv_kernel_size=experiments[i+1]['conv_kernel_size'],pool_size=experiments[i+1]['pool_size'],
                            dropout_rate1=experiments[i+1]['dropout_rate'],dropout_rate2=experiments[i+1]['dropout_rate'],
                            n_conv_layers=experiments[i+1]['n_conv_layers'],conv_nfilters=experiments[i+1]['conv_nfilters'])
    except:
        errCount+=1
        if errCount==1:
            with open('modelfailurelog.txt','w') as f:
                f.write('Experiment {} failed - Check Model Setup'.format(titles[i+1]))
        else:
            with open('modelfailurelog.txt','a') as f:
                f.write('Experiment {} failed - Check Model Setup'.format(titles[i+1]))
    print('Model Created')
    print(model.summary())
    print('Running Experiment-{}'.format(i+1))
    print('---------------------------------------------------------------------')
    try:
        run_experiment(titles[i],model,batch_size=256,epochs=50,df_dict=df_dict,data_augmentation=True)
    except:
        experrCount+=1
        if experrCount==1:
            with open('experimentfailurelog.txt','w') as f:
                f.write('Experiment {} failed - Check Exp Setup'.format(titles[i+1]))
        else:
            with open('experimentfailurelog.txt','a') as f:
                f.write('Experiment {} failed - Check Exp Setup'.format(titles[i+1]))
print('All experiments finished')

Creating Model
Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Model Created
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_layer (InputLayer)        (None, 64, 37, 1)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 64, 37, 100)  1000        input_layer[0][0]                
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 32, 18, 100)  0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 32, 18, 100)  90100       max_pooling2d_1

Model Created
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_layer (InputLayer)        (None, 64, 37, 1)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 64, 37, 100)  1000        input_layer[0][0]                
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 32, 18, 100)  0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 32, 18, 100)  90100       max_pooling2d_1[0][0]            
______________________________________________________________________________

/home/jayanth/anaconda3/envs/tf1_env/lib/python3.6/site-packages/keras_preprocessing/image/image_data_generator.py:341: UserWarning: This ImageDataGenerator specifies `zca_whitening` which overrides setting of`featurewise_std_normalization`.
  warnings.warn('This ImageDataGenerator specifies '


Training model on parquet file #1
-------------------------------------


/home/jayanth/anaconda3/envs/tf1_env/lib/python3.6/site-packages/mlflow/utils/autologging_utils.py:60: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  all_param_names, _, _, all_default_values = inspect.getargspec(fn)  # pylint: disable=W1505


...

/home/jayanth/anaconda3/envs/tf1_env/lib/python3.6/site-packages/keras_preprocessing/image/image_data_generator.py:341: UserWarning: This ImageDataGenerator specifies `zca_whitening` which overrides setting of`featurewise_std_normalization`.
  warnings.warn('This ImageDataGenerator specifies '


Training model on parquet file #1
-------------------------------------


/home/jayanth/anaconda3/envs/tf1_env/lib/python3.6/site-packages/mlflow/utils/autologging_utils.py:60: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  all_param_names, _, _, all_default_values = inspect.getargspec(fn)  # pylint: disable=W1505


Epoch 1/50
176/176 [==============================] - 65s 372ms/step - loss: 8.4727 - output_root_loss: 4.9274 - output_vowel_loss: 2.2137 - output_consonant_loss: 1.3316 - output_root_accuracy: 0.0237 - output_vowel_accuracy: 0.2154 - output_consonant_accuracy: 0.5884 - val_loss: 8.9886 - val_output_root_loss: 5.0046 - val_output_vowel_loss: 2.3934 - val_output_consonant_loss: 1.5897 - val_output_root_accuracy: 1.9916e-04 - val_output_vowel_accuracy: 0.2189 - val_output_consonant_accuracy: 0.1173
Epoch 2/50
176/176 [==============================] - 63s 357ms/step - loss: 7.7906 - output_root_loss: 4.7224 - output_vowel_loss: 1.8944 - output_consonant_loss: 1.1738 - output_root_accuracy: 0.0310 - output_vowel_accuracy: 0.2993 - output_consonant_accuracy: 0.6179 - val_loss: 10.0700 - val_output_root_loss: 5.3397 - val_output_vowel_loss: 3.3568 - val_output_consonant_loss: 1.3752 - val_output_root_accuracy: 0.0116 - val_output_vowel_accuracy: 0.2189 - val_output_consonant_accuracy: 0.62

Compressing Images
Creating Model
Model Created
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_layer (InputLayer)        (None, 64, 37, 1)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 64, 37, 100)  2600        input_layer[0][0]                
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 32, 18, 100)  0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 32, 18, 100)  250100      max_pooling2d_1[0][0]            
____________________________________________